# Реализуем свёрточную нейросеть
В этом упражнении мы реализуем полносвязную нейросеть для классификации и оценим её на датасете CIFAR-10.

In [1]:
# Запустите эту ячейку для проверки работоспособности необходимых модулей.

import numpy as np
import matplotlib.pyplot as plt

from cs231n.classifiers.neural_net import TwoLayerNet

# Немного магии, чтобы заставить графики matplotlib появляться прямо внутри блокнота,
# а не в новом окне.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер графиков по умолчанию
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Больше магии с дополнительными Python-модулями можно найти здесь
# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ относительная ошибка """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

Мы будем использовать класс `TwoLayerNet` из файла `neural_net.py`. Гиперпараметры находятся в переменной `self.params`. Ниже мы инициализируем "игрушечные" данные и модель для реализации нейросети.

In [2]:
# Обратите внимание, что мы устанавливаем random.seed конкретным числом, чтобы повторять эксперименты.

input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5

def init_toy_model():
    np.random.seed(0)
    return TwoLayerNet(input_size, hidden_size, num_classes, std=1e-1)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y

net = init_toy_model()
X, y = init_toy_data()

# Прямой проход: Вычисляем оценки
Откройте файл `neural_net.py` и посмотрите на метод `TwoLayerNet.loss`. Это функция потерь: она берёт данные и веса, а затем вычисляет оценки классов, потери и градиенты. 

Реализуйте первую часть прямого прохода, которая использует весовые коэффициенты и смещения, чтобы вычислить оценки для всех входных данных.

In [ ]:
scores = net.loss(X)
print('Ваши оценки:')
print(scores)
print()
print('Корректные оценки:')
correct_scores = np.asarray([
  [-0.81233741, -1.27654624, -0.70335995],
  [-0.17129677, -1.18803311, -0.47310444],
  [-0.51590475, -1.01354314, -0.8504215 ],
  [-0.15419291, -0.48629638, -0.52901952],
  [-0.00618733, -0.12435261, -0.15226949]])
print(correct_scores)
print()

# Разница должна быть очень маленькой, < 1e-7
print('Разница между вашими оценками и корректными оценками:')
print(np.sum(np.abs(scores - correct_scores)))

# Прямой проход: вычисляем потери
В той же функции реализуйте вторую часть, которая вычисляет потери данных и регуляризации.

In [ ]:
loss, _ = net.loss(X, y, reg=0.05)
correct_loss = 1.30378789133

# Должна быть очень маленькой, < 1e-12
print('Разница между вашими оценками и корректными оценками:')
print(np.sum(np.abs(loss - correct_loss)))

# Обратный проход
Реализуйте остальную часть функции. Она должна вычислить градиент потерь по отношению к переменным `W1`, `b1`, `W2` и `b2`. Если вы правильно реализовали прямой проход, вы можете проверить обратный проход с помощью числового градиента:

In [ ]:
from cs231n.gradient_check import eval_numerical_gradient

# Используйте числовую проверку.
# Если ваша реализация правильная, то разница между числовым и аналитическим градиентом должна быть
# менее чем 1e-8 для всех переменных: W1, W2, b1 и b2.

loss, grads = net.loss(X, y, reg=0.05)

# Все должны быть меньше 1e-8
for param_name in grads:
    f = lambda W: net.loss(X, y, reg=0.05)[0]
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    print('%s максимальная относительная погрешность: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

# Обучаем нейросеть
Для обучения нейросети мы используем стохастический градиентный спуск (SGD). Посмотрите на функцию `TwoLayerNet.train` и заполните пропуски, чтобы реализовать процесс обучения. Вам также необходимо будет написать метод `TwoLayerNet.predict`, поскольку процесс обучения периодически выполняет прогнозы, чтобы отслеживать точность во время обучения.

После этого запустите код ниже, чтобы обучить простую нейросеть. Вы должны получить потери обучения меньше 0.02.

In [ ]:
net = init_toy_model()
stats = net.train(X, y, X, y,
            learning_rate=1e-1, reg=5e-6,
            num_iters=100, verbose=False)

print('Финальные потери обучения: ', stats['loss_history'][-1])

# Выводим график потерь
plt.plot(stats['loss_history'])
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

# Загружаем данные
Теперь, когда вы реализовали двухслойную нейросеть, которая проходит градиентную проверку, пришло время загрузить ваш любимый CIFAR-10 и обучить нейросеть на реальных данных.

In [ ]:
from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    """
    Загружаем датасет CIFAR-10 и выполняем предварительную обработку данных.  
    """
    # Загрузка данных
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    # Очищаем переменные, чтобы данные можно было загружать много раз 
    try:
       del X_train, y_train
       del X_test, y_test
       print('Очистка предыдущих данных.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
        
    # Разделяем данные на подвыборки
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Нормализуем данные
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    # Превращаем данные в строки
    X_train = X_train.reshape(num_training, -1)
    X_val = X_val.reshape(num_validation, -1)
    X_test = X_test.reshape(num_test, -1)

    return X_train, y_train, X_val, y_val, X_test, y_test


# Выводим размеры данных
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Объём обучающей выборки: ', X_train.shape)
print('Число обучающих меток: ', y_train.shape)
print('Объём оценочной выборки: ', X_val.shape)
print('Число оценочных меток: ', y_val.shape)
print('Объём тестовой выборки: ', X_test.shape)
print('Число тестовых меток: ', y_test.shape)

# Обучаем нейросеть
Для обучения мы снова будем использовать SGD. Кроме того, мы будем корректировать скорость обучения с помощью экспоненциального графика по мере оптимизации; после каждой эпохи мы будем снижать скорость обучения, умножая ее на скорость затухания.

In [ ]:
input_size = 32 * 32 * 3
hidden_size = 50
num_classes = 10
net = TwoLayerNet(input_size, hidden_size, num_classes)

# Обучаем нейросеть
stats = net.train(X_train, y_train, X_val, y_val,
            num_iters=1000, batch_size=200,
            learning_rate=1e-4, learning_rate_decay=0.95,
            reg=0.25, verbose=True)

# Делаем прогнозы на оценочной выборке
val_acc = (net.predict(X_val) == y_val).mean()
print('Точность оценки: ', val_acc)


# Отлаживаем обучение
С параметрами по умолчанию, которые мы указали выше, вы должны получить точность около 0.29 для оценочной выборки. Это не очень хороший результат.

Одна из стратегий, позволяющая понять, что не так - построить график функции потерь и точности для обучающей и оценочной выборок при оптимизации.

Другая стратегия заключается в визуализации весов, которые были изучены на первом слое сети.

In [ ]:
# Выводим график функции потерь и точности
plt.subplot(2, 1, 1)
plt.plot(stats['loss_history'])
plt.title('Loss history')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.subplot(2, 1, 2)
plt.plot(stats['train_acc_history'], label='train')
plt.plot(stats['val_acc_history'], label='val')
plt.title('Classification accuracy history')
plt.xlabel('Epoch')
plt.ylabel('Classification accuracy')
plt.legend()
plt.show()

In [ ]:
from cs231n.vis_utils import visualize_grid

# Визуализируем веса

def show_net_weights(net):
    W1 = net.params['W1']
    W1 = W1.reshape(32, 32, 3, -1).transpose(3, 0, 1, 2)
    plt.imshow(visualize_grid(W1, padding=3).astype('uint8'))
    plt.gca().axis('off')
    plt.show()

show_net_weights(net)

# Настраиваем гиперпараметры

**В чём дело?** Глядя на приведенные выше визуализации, мы видим, что потери уменьшаются более или менее линейно - это может говорить о том, что скорость обучения слишком низкая. Более того, нет никакой разницы между точностью обучения и оценки. Значит, используемая нами модель слишком мала. С другой стороны, с большой моделью можно столкнуться с переобучением, которое проявится в очень большом разрыве между точностью обучения и оценки.

**Настройка**: Настройка гиперпараметров и оценка того, как они будут влиять на эффективность модели - важная часть процесса обучения нейросетей, которую стоит регулярно практиковать. Ниже вы должны поэкспериментировать с различными значениями гиперпараметров, включая размер скрытого слоя, скорость обучения, количество эпох обучения и силу регуляризации. Вы также можете подумать о настройке снижения скорости обучения, но неплохой точности можно добиться и со значением по умолчанию.

**Приблизительные результаты**: Вы должны стремиться к достижению точности классификации более 48% на оценочной выборке. 52% - в данном случае хороший результат.

**Эксперимент**: Ваша цель в этом упражнении - получить как можно лучший результат на CIFAR-10. Не стесняйтесь использовать свои собственные методы.

In [ ]:
best_net = None # сохраните сюда лучшую модель

#################################################################################
# TODO: Настроить гиперпараметры с помощью оценочной выборки и сохранить модель #
# в переменную best_net.                                                        #
#                                                                               #
# Полезно будет использовать здесь те же графики, что мы применяли выше.        #
# Они должны существенно отличаться от плохо настроенной нейросети.             #
#                                                                               #
# Лучше всего написать код для автоматического подбора гиперпараметров, чем     #
# перебирать их вручную.                                                        #
#################################################################################
# *****START OF YOUR CODE*****

pass

# *****END OF YOUR CODE*****


In [ ]:
# Визуализация весов лучшей нейросети
show_net_weights(best_net)

# Запускаем сеть на тестовой выборке
Когда закончите с экспериментами, оцените финальную нейросеть на тестовой выборке. Вы должны получить точность выше 48%.

In [ ]:
test_acc = (best_net.predict(X_test) == y_test).mean()
print('Точность на тестовой выборке: ', test_acc)